In [1]:
#Compute limits
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan  9 04:33:10 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   35C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Load the Base and Finetuned Model

In [2]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install rouge_score
!pip install peft
!pip install trl
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=71ff7fd50ffbbb4ea9870638

In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import statistics
import json
import os
import transformers
import torch
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [5]:
# Download Gemma 2b base model
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config = bnb_config,
                                            device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
# Unzip the Lora finetuned model
!unzip -o '/content/gemma_2b_Lora_finetuned_telugu_story_telling.zip' -d 'gemma_2b_Lora_finetuned_telugu_story_telling'

Archive:  /content/gemma_2b_Lora_finetuned_telugu_story_telling.zip
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/tokenizer.model  
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/README.md  
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/special_tokens_map.json  
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/adapter_model.safetensors  
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/tokenizer.json  
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/tokenizer_config.json  
  inflating: gemma_2b_Lora_finetuned_telugu_story_telling/adapter_config.json  


In [7]:
# Load the LoRA fine-tuned weights on top of the base model.
lora_model_path = "/content/gemma_2b_Lora_finetuned_telugu_story_telling"
model = PeftModel.from_pretrained(base_model, lora_model_path)

# Performance Evaluation

In [27]:
# Load the test dataset
df_test = pd.read_csv('/content/test.csv')

In [28]:
df_test

,Prompt,Title,Story
0,వృక్షం గురించి కథ చెప్పు?,సజీవ దేవుడు,భర్త రాము పనీపాటా లేకుండా తోటలో కూర్చుని ఉండటం...
1,లోభం గురించి కథ చెప్పు?,కపట దానం,పూర్వం చంద్రనగరంలో ఒక కరణం ఉండే వాడు. ఆయనకు అం...
2,బామ్మ భీమన్న కథ చెప్పు?,పళ్ళబుట్ట గొడవ,బామ్మ ఒకనాడు గ్రామాధికారి ఇంటికి వెళ్ళి ఆయనతో ...
3,కళింగ యుద్ధతంత్రం గురించి కథ చెప్పు?,యుద్ధతంత్రం,\nకళింగ దేశాన్ని పరిపాలించే చంద్రహాసుడు విహారయ...
4,మనసులోని మర్మం కథ చెప్పు?,మనసులోని మర్మం,ఆరావళి పర్వత ప్రాంతాన్ని ఆనుకుని ఉన్న రాజ్యాన్...
5,భీమన్న వర్తకులు కథ చెప్పు?,కొత్త నౌకరి,షావుకారిచ్చిన పావలా జేబులో వేసుకుని భీమన్న నడక...
6,పిశాచం గురించి కథ చెప్పు?,పిశాచం వదిలింది,"గుండు భీమన్న ఒక ఇంటివాడై, తన భార్య అయిన మహలక్ష..."
7,యుక్తి శక్తి గురించి కథ చెప్పు?,యుక్తి వేరు శక్తి వేరు,పెద్దబ్బాయికి పరీక్షలు దగ్గిరికొస్తున్నాయి. కా...
8,వంట గురించి కథ చెప్పు?,వంటల రాణి,ఓసారి శ్రీ కృష్ణదేవరాయలు మారువేషంలో నగరంలో తిర...
9,తీర్పు గురించి ఒక కథ చెప్పు?,తీర్పు,నీరవుడు తన పెరటిలో నాటిన కొబ్బరి మొక్క ఏపుగా ఎ...


In [8]:
def generate_chunk(prompt, by_user=0):
  input_ids = tokenizer.encode("prompt:" + prompt + "\n", return_tensors='pt').to('cuda')
  attention_mask = torch.ones(input_ids.shape).to('cuda')
  attention_mask = torch.ones(input_ids.shape).to('cuda')
  # Generate text deterministically
  output = model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=1024,)
#    temperature=0.0,   # Set temperature to 0 for deterministic output
#    top_k=1,           # Consider only the top token
#    top_p=0.0,         # Disable nucleus sampling
#    do_sample=False,   # Disable sampling to use greedy decoding
#)
  gen_text = tokenizer.decode(output[0], skip_special_tokens=True)

  if by_user == 1:
    prompt, gen_chunk = gen_text.split('Title:', maxsplit=1)
  else:
    prompt, gen_chunk = gen_text.split(prompt, maxsplit=1)
  return gen_chunk

In [9]:
def generate_full_story(user_prompt):
    story = 'Title:'
    story += generate_chunk(user_prompt, by_user=1)

    next_prompt = ''.join([l.strip() + '. ' for l in story.split('.')[-2:-1]])
    i = 1; max_generations = 9 # The longest story in trainset has 9 parts.

    while story[-11:] != 'కథ సమాప్తం.':
      if i > max_generations:
        break
      gen_chunk = generate_chunk(next_prompt)
      i = i + 1
      story = story + '\n' + gen_chunk
      next_prompt = ''.join([l.strip() + '. ' for l in gen_chunk.split('.')[-2:-1]])

    return story


In [31]:
model.eval()
df_test['generated_story'] = None
df_test['generated_story'] = df_test['Prompt'].apply(generate_full_story)

In [32]:
# custom implementation for ROUGE-1, ROUGE-2, and ROUGE-L. This code calculates precision, recall, and F1-score for each metric.
# The rouge_score libarries had issues calculating scores for telugu text, hence using a custom implementation

def compute_rouge(reference, generated):
    def tokenize(text):
        return text.split()

    def ngrams(tokens, n):
        return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

    def compute_overlap(set_ref, set_gen):
        overlap = set_ref & set_gen
        precision = len(overlap) / len(set_gen) if set_gen else 0
        recall = len(overlap) / len(set_ref) if set_ref else 0
        f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
        return {"precision": precision, "recall": recall, "f1": f1}

    # Tokenize texts
    ref_tokens = tokenize(reference)
    gen_tokens = tokenize(generated)

    # ROUGE-1 (Unigrams)
    ref_unigrams = set(ngrams(ref_tokens, 1))
    gen_unigrams = set(ngrams(gen_tokens, 1))
    rouge1 = compute_overlap(ref_unigrams, gen_unigrams)

    # ROUGE-2 (Bigrams)
    ref_bigrams = set(ngrams(ref_tokens, 2))
    gen_bigrams = set(ngrams(gen_tokens, 2))
    rouge2 = compute_overlap(ref_bigrams, gen_bigrams)

    # ROUGE-L (Longest Common Subsequence)
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i - 1] == Y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
        return dp[m][n]

    lcs_length = lcs(ref_tokens, gen_tokens)
    recall_lcs = lcs_length / len(ref_tokens) if ref_tokens else 0
    precision_lcs = lcs_length / len(gen_tokens) if gen_tokens else 0
    f1_lcs = (2 * precision_lcs * recall_lcs / (precision_lcs + recall_lcs)) if (precision_lcs + recall_lcs) > 0 else 0
    rougeL = {"precision": precision_lcs, "recall": recall_lcs, "f1": f1_lcs}

    return {"rouge1": rouge1, "rouge2": rouge2, "rougeL": rougeL}

In [33]:
# Calculate ROUGE scores
rouge_scores = []
for index, row in df_test.iterrows():
    target_story = 'Title: ' + row['Title'] + '\n' + 'Story: ' + row['Story']
    generated_story = row['generated_story']
    scores =compute_rouge(target_story, generated_story)
    rouge_scores.append(scores)

# Print average ROUGE scores
avg_rouge1 = sum([score['rouge1']['f1'] for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2']['f1'] for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL']['f1'] for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 Score: {avg_rouge1}")
print(f"Average ROUGE-2 Score: {avg_rouge2}")
print(f"Average ROUGE-L Score: {avg_rougeL}")

Average ROUGE-1 Score: 0.5664814143228628
Average ROUGE-2 Score: 0.5134189293820322
Average ROUGE-L Score: 0.5339462762073938


In [34]:
df_test # staring down at actual and generated story

,Prompt,Title,Story,generated_story
0,వృక్షం గురించి కథ చెప్పు?,సజీవ దేవుడు,భర్త రాము పనీపాటా లేకుండా తోటలో కూర్చుని ఉండటం...,Title: సజీవ దేవుడు\nStory: భర్త రాము పనీపాటా ల...
1,లోభం గురించి కథ చెప్పు?,కపట దానం,పూర్వం చంద్రనగరంలో ఒక కరణం ఉండే వాడు. ఆయనకు అం...,Title: కపట దానం\nStory: పూర్వం చంద్రనగరంలో ఒక ...
2,బామ్మ భీమన్న కథ చెప్పు?,పళ్ళబుట్ట గొడవ,బామ్మ ఒకనాడు గ్రామాధికారి ఇంటికి వెళ్ళి ఆయనతో ...,Title: పిశాచం వదిలింది\nStory: గుండు భీమన్న ఒక...
3,కళింగ యుద్ధతంత్రం గురించి కథ చెప్పు?,యుద్ధతంత్రం,\nకళింగ దేశాన్ని పరిపాలించే చంద్రహాసుడు విహారయ...,Title:కళింగ పర్వతాలు\nStory:కళింగనగరం అనే రాజ్...
4,మనసులోని మర్మం కథ చెప్పు?,మనసులోని మర్మం,ఆరావళి పర్వత ప్రాంతాన్ని ఆనుకుని ఉన్న రాజ్యాన్...,Title:మనమే మర్మం\nStory: ఆరావళి పర్వత ప్రాంతాన...
5,భీమన్న వర్తకులు కథ చెప్పు?,కొత్త నౌకరి,షావుకారిచ్చిన పావలా జేబులో వేసుకుని భీమన్న నడక...,Title:గ్రహదోషం\nStory:జమీందారుగారి అసంతృప్తి మ...
6,పిశాచం గురించి కథ చెప్పు?,పిశాచం వదిలింది,"గుండు భీమన్న ఒక ఇంటివాడై, తన భార్య అయిన మహలక్ష...",Title: చిత్రకారుల యుక్తి\nStory: పూర్వం జయవిజయ...
7,యుక్తి శక్తి గురించి కథ చెప్పు?,యుక్తి వేరు శక్తి వేరు,పెద్దబ్బాయికి పరీక్షలు దగ్గిరికొస్తున్నాయి. కా...,Title: యుక్తి శక్తి\nStory: పెద్దబ్బాయికి పరీక...
8,వంట గురించి కథ చెప్పు?,వంటల రాణి,ఓసారి శ్రీ కృష్ణదేవరాయలు మారువేషంలో నగరంలో తిర...,Title: వంటల రాణి\nStory: ఓసారి శ్రీ కృష్ణదేవరా...
9,తీర్పు గురించి ఒక కథ చెప్పు?,తీర్పు,నీరవుడు తన పెరటిలో నాటిన కొబ్బరి మొక్క ఏపుగా ఎ...,Title: తీర్పు\nStory: నీరవుడు తన పెరటిలో నాటిన...


In [35]:
df_test.to_csv('output.csv', index=False)

In [36]:
# Human Evaluation
model.eval()
user_prompt = "బ్రహ్మప్రళయం గురించి ఒక చందమామ కథ చెప్పు?"
generated_story = generate_full_story(user_prompt)
print("GENERATED STORY:\n" + generated_story)

GENERATED STORY:
Title:బ్రహ్మ ప్రళయం
Story: బ్రహ్మదేవుడు భూమిపై మానవులను సృష్టించి, నాయనలారా, తిరిగి నేను ప్రళయం కలిగించేదాకా మీరు ఈ భూమిపై జీవించండి అని వరం ఇచ్చాడు.
దేవా తిరిగి ప్రళయం, ఎప్పుడు వస్తుందో మాకు తెలిసినట్టయితే మేము భూమిపై చేయదగిన కార్యాలను నిర్ణయించుకుంటాము అన్నారు మానవులు.
ప్రళయం ఎంతకాలానికి జరిగేదీ మీకు తెలిసేందుకు నేనొక ఏర్పాటు చేస్తాను, అంటూ బ్రహ్మదేవుడు ఒకచోట మూడు కర్రలు పాతాడు. మొదటి కరకు వలయాలు అమర్చాడు. అట్టడుగున ఉన్న వలయం అన్నిటి కన్నా పెద్దది, దానిపైది కొంచెం చిన్నది, దానిపైది ఇంకా కొంచెం చిన్నది, అన్ని వలయాలకూ పైన ఉన్నది అన్నిటికన్న చిన్నది.
తరవాత బ్రహ్మదేవుడు ముగ్గురినీ పిలిచి, మీరు బ్రహ్మప్రళయం దాకా జీవించే వరం ఇస్తున్నాను. మీ పని ఏమిటంటే ఈ వలయాలన్నిటినీ ఇదే క్రమంలో మూడవ కర్రకు మార్చాలి. పెద్ద వలయం మీద చిన్న వలయం ఉంచవచ్చునే గాని చిన్నదాని మీద పెద్ద వలయం ఉంచరాదు. రెండవ కర్రను తాత్కాలికంగా వలయాలుంచటానికి మాత్రవే ఉపయోగించాలి. మీరు ముగ్గురూ వంతులు వేసుకుని ఈ పని సాగించండి. ఈ అరవై నాలుగు వలయాలూ మూడవ కర్రకు ఇదే క్రమంలో అమర్చిన క్షణాన ప్రళయం వస్తుంది అన్నాడు. మరుక్ష

# Inference

In [10]:
# Pre-requisites for Inferece:
  # -- Run all the cells in "Load the Base and Finetuned Model" section
  # -- Run helper functions "generate_chunk" and "generate_full_story" in "Performance Evaluation" section.

# You can run inference by giving the prompts to the model as shown the examples below.
prompt_examples = """
1. శంకరాచార్యుల గురించి  ఏదైనా కథ చెప్పు?
2. మోసం చేసే బాబాలు గురించి కథ చెప్పు?
3. ఒక మంచి రాజు కథ చెప్పు?
4. తెలివైన అమ్మాయి గురించి ఒక కథ చెప్పు?
5. చెడ్డ వాళ్ల స్నేహం గురించి కథ చెప్పు?
6. ఏదైనా తమాషా కథ చెప్పు?
"""

In [11]:
model.eval() # set model to evaluation mode.

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [13]:
user_prompt = "శంకరాచార్యుల గురించి  ఏదైనా కథ చెప్పు?"
generated_story = generate_full_story(user_prompt)
print("GENERATED STORY:\n" + generated_story)

GENERATED STORY:
Title: రెండు రహస్యాలు
Story: జగద్గురువు ఆదిశంకరాచార్యుల వారు దేశమంతటా పాదయాత్రలు చేస్తూ భగవంతుని సారాంశాన్ని వ్యాపింపచేస్తున్న రోజులవి. అలా ఓ రోజు అటవీ ప్రాంతం గుండా కాశీనగరానికి తన భక్త బృందంతో యాత్రసాగిస్తూ ఉండగా, చీకటిపడే సమయం కావొచ్చింది.
దూరంగా మినుకు మినుకుమంటూ దీపపు కాంతులతో ఓ ఊరు కానరావడంతో ఆ పూటకు అక్కడికి చేరి కాస్త విశ్రమించి మరునాడు పయనం సాగించవచ్చని శంకరాచార్యుల వారు శిష్యులకు ఆనతీయడంతో వారంతా ఊరి వైపు పయనం సాగిం చారు.
ఊరు సమీపించగానే ఆది శంకరాచార్యులవారు విచ్చేస్తున్నారని తెలిసిన జనులు తండోపతండాలుగా చేరి భజనలతో, దీపాలతో, పుష్పాలతో ఎదురేగి వచ్చి వారందరినీ ఊరిలో ఆశ్రమానికి కొనిపోయారు. కావలసిన పాలు పండ్లు భోజభక్ష్యాదులు అమర్చి ఆ రాత్రి వారిని సేవించారు.
మర్నాడు ఊరిలోని ప్రతి గడపను దర్శించారు, ఇంటింటో గోమాత దర్శనమిచ్చింది. ప్రతి ఇంటి వాకిటా తులసి వనాలు దర్శనమిచ్చాయి. పూజా మందిరాలలో వేదాలు వల్లింపబడుతున్నాయి. ఆ ఊరిని చూసేసరికి సమస్త దేవతలూ అక్కడే కొలువుతీరి ఉన్నట్లుగ్గా అనిపించింది.
జనులందరినీ దీవిస్తూ జగద్గురువు తన శిష్యులతో తిరిగి పయనమై అందరి వద్దా వీడ్కోలు 